In [54]:
# Import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [59]:
#import topuni data
topuni_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508627223168'
request = requests.get(topuni_url)
topuni_data = request.json()['data']

#add data to dataframe
topuni_df = pd.io.json.json_normalize(topuni_data)

#trim data frame to top 200 universities
topuni_df = topuni_df.head(200)

#drop columns that we don't need
topuni_df = topuni_df.drop(["cc", "core_id","logo","guide","nid","score","stars"],axis=1)

topuni_df

,country,rank_display,region,title,url
0,United States,1,North America,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,United States,2,North America,Stanford University,/universities/stanford-university
2,United States,3,North America,Harvard University,/universities/harvard-university
3,United States,4,North America,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,United Kingdom,5,Europe,University of Cambridge,/universities/university-cambridge
5,United Kingdom,6,Europe,University of Oxford,/universities/university-oxford
6,United Kingdom,7,Europe,UCL (University College London),/universities/ucl-university-college-london
7,United Kingdom,8,Europe,Imperial College London,/universities/imperial-college-london
8,United States,9,North America,University of Chicago,/universities/university-chicago
9,Switzerland,10,Europe,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


In [61]:
#import timesedu data
timesedu_url = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
request = requests.get(timesedu_url)
timesedu_data = request.json()['data']

#add data to dataframe
timesedu_df = pd.io.json.json_normalize(timesedu_data)

#trim data frame to top 200 universities
timesedu_df = timesedu_df.head(200)

#drop columns that we don't need
timesedu_df = timesedu_df.drop(["member_level","nid","record_type"],axis=1)

timesedu_df


#topuniversities
# URL = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2018'
# r = requests.get(URL)
# page_body = r.text
# soup = BeautifulSoup(page_body, 'html.parser')
# soup.prettify
# table = soup.find_all('table', id = 'qs-rankings')
# table

,aliases,location,name,rank,rank_order,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,University of Oxford,1,10,99.1,15,63.7,169,95.0,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,University of Cambridge,2,20,97.5,29,51.5,260,93.0,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,California Institute of Technology,=3,30,99.5,10,92.6,51,59.7,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,Stanford University,=3,40,99.9,4,60.5,189,77.6,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,Massachusetts Institute of Technology,5,50,100.0,1,88.4,63,87.6,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,Harvard University,6,60,99.7,8,46.4,330,79.7,...,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,Princeton University,7,70,99.6,9,58.0,204,78.7,...,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,Imperial College London,8,80,96.7,40,71.6,120,96.6,...,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,University of Chicago,9,90,99.4,12,39.8,487,69.6,...,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,ETH Zurich – Swiss Federal Institute of Techno...,=10,100,94.3,60,60.3,190,98.1,...,92.0,8,76.4,21,31 : 69,"19,233",38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...
